# National tourism promotion
## Group project
### This notebook uses the *EuropeTop100Attractions_ENG_20190101_20210821* and the *Holidays.csv* datasets

(c) Nuno António 2021 - Rev. 1.01

## *EuropeTop100Attractions_ENG_20190101_20210821* dataset description

### Sheet **Reviews**
Reviews published in Tripadvisor from January 1, 2019 to August 21, 2021, in English, for the top 100 tourist attractions in Europe.
- **localID**: string - ID of the attraction
- **extractionDate** - date - date when the review was extracted
- **globalRating** - numeric - global rating of the attraction at the time of the review extraction (reviews in Tripadvidor are in a scale from 1 to 5 stars)
- **positionOnRanking** - numeric - position in TripAdvisor's regional ranking at the extraction date
- **sitesOnRanking** - numeric - total number of attractions in TripAdvisor's regional ranking at the extraction date
- **totalReviews** - numeric - total reviews written for the attraction at the time of the review extraction
- **userName** - string - user name of the TripAdvisor user who posted the review. The user name is composed of two parts (first@second). The first is the public name of the user. The second is the TripAdvisor unique identifier of the user.
- **userLocation** - string - location of where the user who posted the review lives. This is not a mandatory field, so many users to not provide their location
- **userContributions** - numeric - how many reviews have the user wrote in TripAdvisor at the moment of the extraction of the review
- **tripType** - string - type of trip type. This is not a mandatory field
- **reviewWritten** - date - date when the review was published
- **reviewVisited** - date - date when the customer visited the attraction. The day is always 1 because Tripadvisor only ask users to describe the year and the month, not the day
- **reviewRating** - numeric - quantitative rating assigned by the user (1 star - bad to 5 stars - excellent)
- **reviewLanguage** - string - language the review was written (in this case should be always "en" for english)
- **reviewFullText** - string - full text of the review (since this course does not address Text Mining the use of this field is completely optional and its use will not be considered for grading)


### Sheet **Attractions**
Information about the attractions.
- **ID**: string - ID of the attraction
- **Name**: string - name of the attraction
- **Country**: string - name of the country or region
- **ISO**: string - ISO code of the country or region

## *Holidays.csv* dataset description
Worldwide public holidays. Additional information available at https://docs.microsoft.com/en-us/azure/open-datasets/dataset-public-holidays?tabs=azure-storage#data-access
- **countryOrRegion**: string - country or region full name
- **countryOrRegionCode**: string - country or region in ISO format
- **date**: date - date of the holiday
- **holidayName**: string - full name of the holiday
- **isPaidTimeOff**: boolean - indicates whether most people have paid time off on this date (only available for US, GB, and India now). If it is NULL, it means unknown
- **normalizeHolidayName**: normalized name of the holiday

## Group details
- Composed of three students. Groups of two are aceptable, but must be approved by instructors.
- Students can be from different theory and practical classes.

## Work description

### Overview
<p>Tourism was hit hard by the COVID-19 pandemic. National Tourism Board Organizations (NTBO) want to study users' generated content, such as Social Media reviews, to understand visitors patterns and if these patterns were affected by the pandemic.</p>
<p>Employing the CRISP-DM process model, your group are expected to act as a consultant for the Portuguese NTBO. Your job is to characterize and describe the patterns of visitants of Portuguese attractions and comparing it to Portugal's main tourism competitors. In addition, you can try to understand visitors' frequent itemset associations, similarities between attractions or visitors, or segment visitors using RFM (replacing Monetization by ratings for example).</p>

Examples of the type of analysis that can be done:
- Comparison of reviews frequency and rating distribution before and after the pandemic
- Comparison of the type of trip and origin of visitors

<p>Invest time in explaining the rationale of your choices and your business recommendations based on your findings.</p>

### Deliverables
- Python source code (Jupyter notebook or .py files) and/or Excel files. Python code should be commented to facilitate comprehension
- Powerpoint presentation
- Report:
    - Maximum of 20 pages (excluding appendixes)
    - Minimum font size is 10
    - Should describe the main outputs according to CRISP-DM, including the brief description of the problem, methods, results, and their discussion


### Presentation
- To be done in the exam season with all group members present
- Slots of 20 minutes per group
- 10 minutes for presentation, another 10 for discussion


### Questions or additional informations
For any additional questions, don't hesitate to get in touch with the instructors of the practical classes. They will also act as the national tourism board business/project stakeholders.

<br><br>
Good work or good luck ;)

# BUSINESS UNDERSTANDING (business objectives, assess situation, DM goals, Produce project plan)

Business understanding section explained in the report

# DATA LOADING & UNDERSTANDING (collect, describe, explore, verify)

## Data collection

In [6]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import seaborn as sea
import openpyxl
from pathlib import Path
import numpy as np
import joypy
import re
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules
import networkx as nx
from sklearn.metrics import pairwise_distances
import matplotlib.dates as mdates
from matplotlib import ticker
from mpl_toolkits.mplot3d import Axes3D
from matplotlib import cm
import ipympl
from mpl_toolkits.mplot3d import Axes3D
import squarify
a4_dims = (20, 12)
from matplotlib import pyplot

# import of libraries needed for this project

In [7]:
subPlots_Title_fontSize = 12
subPlots_xAxis_fontSize = 10
subPlots_yAxis_fontSize = 10
subPlots_label_fontSize = 10
heatmaps_text_fontSize = 8

plots_Title_fontSize = 14
plots_Title_textColour = 'black'

plots_Legend_fontSize = 12
plots_Legend_textColour = 'black'

In [8]:
# Load attraction data 
ds_attractions = pd.read_excel('EuropeTop100Attractions.xlsx')

# This excel is provided by the team via moodle submission
# We split each sheet from the 100attractions excel file into separate "excel" - so the neccessary merging of tables we do here in Python


Also, prior loading attraction data above - our team has made a quick checkup of data accuracy through pivot tables. 

Some basic crosschecks of duplicated names and country names have been investigated and corrected. Following actions have been done just to save time and effort that can be used in further analysis and data cleaning. Please, follow the actions we have taken prior ds_attractions:

- 2x same name "Old Town" --> treated as Warsaw and Dubrownik Old Towns after double check through reviews escel sheet
- MAG045 (Old town) that has been renamed to Warsaw Old Town has been given wrong ISO (HR) instead of PL. This has been corrected.
- Vatican has been assigned to italy but with VA ISO code and not italy. We changed it to Italian ISO code
- Scot and Scotland were like two countries, even though they are obviously one. We changed Scot to Scotland. With this, as Scotland belongs to the UK, we expect from the data description to have more countries than ISO codes. UK ISO code can hold England too for ex.




In [11]:
# renaming ID column to the same column name as in other excel file to have a succesfull merge of two sheets
ds_attractions.rename(columns = {'ID':'localID'}, inplace = True)

In [10]:
# Check whether Old Town is not duplicated anymore since the analysis (through pivot table in Excel showed two equal values one in Warsaw and one in Durbovnik with the same respective names)
ds_attractions[ds_attractions['Name'] == 'Old Town']

# this, as explained before, has been issued in the given excel sheet - this is just a true crosscheck

,ID,Name,Country,ISO


In [9]:
ds_attractions.head(5)

,ID,Name,Country,ISO
0,MAG001,Basilica of the Sagrada Familia,Spain,ES
1,MAG002,Parc Guell,Spain,ES
2,MAG003,Tower of London,England,UK
3,MAG004,Casa Batllo,Spain,ES
4,MAG005,Staromestske namesti,Czech Republic,CZ


In [13]:
# Load reviews sheet data
ds_reviews = pd.read_excel('EuropeTop100Attractions_reviews.xlsx')

# this excel file is also provided by our team via moodle submission

In [12]:
# Merging two sheets of the excel file into the one based on common ID - localID
ds_one = ds_reviews.merge(ds_attractions, on='localID', how='left')

# source: https://towardsdatascience.com/left-join-with-pandas-data-frames-in-python-c29c85089ba4

NameError: name 'ds_reviews' is not defined

In [ ]:
ds_one.head()

## Data description report

In [ ]:
ds_one.describe(include='all').T

General observations: 
- 92120 rows (records/ratings)
- We have some missing values on User Location, triptypes, Name (together witg Country and ISO)
- why only 98 unique names --> should be 100 as Local IDs
- ISO does not match as country Scotland belongs to the UK, same as England
- Too many user locations

more in depth explanation provided in the report


In [ ]:
ds_one["Country"].value_counts(ascending=False).to_frame()

## Data exploration

In [ ]:
# Most rated countries - ordered ascendingly (count)
plt.figure(figsize=(15,7))
ax = sns.countplot(x='Country', data=ds_one, order=pd.value_counts(ds_one['Country']).iloc[:25].index);
plt.title('Nr of ratings per country')
ax.set_xticklabels(ax.get_xticklabels(), rotation=90, ha="right")
plt.tight_layout()
plt.show()

In [ ]:
# top 20 visited (reviewed) attractions
plt.figure(figsize=(15,7))
ax = sns.countplot(x='Name', data=ds_one, order=pd.value_counts(ds_one['Name']).iloc[:20].index);
plt.title('Top 20 most visited places in Europe')
ax.set_xticklabels(ax.get_xticklabels(), rotation=90, ha="right")
plt.tight_layout()
plt.show()

In [ ]:
# Draw triptypes across all countries in ascending value
plt.figure(figsize=(14,7))
ax = sns.countplot(x='tripType', data=ds_one, order=pd.value_counts(ds_one['tripType']).iloc[:20].index);
plt.title('Trip types ordered ascendignly')
ax.set_xticklabels(ax.get_xticklabels(), rotation=90, ha="right")
plt.tight_layout()
plt.show()

In [ ]:
# Top user locations (where users come from based on nr of ratings provided)
plt.figure(figsize=(15,7))
ax = sns.countplot(x='userLocation', data=ds_one, order=pd.value_counts(ds_one['userLocation']).iloc[:50].index);
plt.title('Most user locations')
ax.set_xticklabels(ax.get_xticklabels(), rotation=90, ha="right")
plt.tight_layout()
plt.show()

# London wins by far, yet we can see that userlocation is so broken based on hundreds of values. Yet, it can be visible that most from users come from either UK or US or Australia. We will go into that more deeper

In [ ]:
# hist (count)plot on the whole dataset with reviewWritten

sns.displot(data=ds_one, x='reviewWritten', kind='hist', height=6, aspect=1.4, bins=35)

# setting customized ticklabels for x axis
pos = ['2019-01-01','2019-02-01','2019-03-01','2019-04-01','2019-05-01','2019-06-01','2019-07-01','2019-08-01','2019-09-01',
      '2019-10-01','2019-11-01','2019-12-01','2020-01-01','2020-02-01','2020-03-01','2020-04-01','2020-05-01','2020-06-01'
      ,'2020-07-01','2020-08-01','2020-09-01','2020-10-01','2020-11-01','2020-12-01','2021-01-01','2021-02-01','2021-03-01',
      '2021-04-01','2021-05-01','2021-06-01','2021-07-01','2021-08-01']
  
lab = ['Jan19',
      'Feb19', 'Mar19', 'Apr19', 'May19', 'June19', 
       'July19', 'Aug19', 'Sept19', 'Oct19', 'Nov19', 'Dec19','Jan20','Feb20', 'Mar20', 'Apr20', 'May20', 'June20', 
       'July20', 'Aug20', 'Sept20', 'Oct20', 'Nov20', 'Dec20','Jan21','Feb21', 'Mar21', 'Apr21', 'May21', 'June21', 
       'July21', 'Aug21']

plt.xticks( pos, lab)
plt.xticks(rotation=45)
plt.show()

# There is a huge drop after Feb2020, meaning Covid has probably hit hard visitance of attractions across all places.
# We explore that further only for Portugal and its closest countrioes (Spain, France, italy) through density plot (below)

In [ ]:
# DENSITY PLOT (Kernel Density Estimate) Portugal vs Rest of countries

fig, ax = plt.subplots(figsize=(15,7))

# sns.kdeplot(portugal_two[portugal_two['Updated_location']=='United States']['reviewVisited'], shade=False, color='yellow', x='reviewVisited')
# sns.kdeplot(portugal_two[portugal_two['Updated_location']=='United Kingdom']['reviewVisited'], shade=False, color='red', x='reviewVisited')
sns.kdeplot(ds_one[ds_one['Country']=='Portugal']['reviewWritten'], shade=False, color='green', x='reviewVisited')
sns.kdeplot(ds_one[ds_one['Country']!='Portugal']['reviewWritten'], shade=False, color='blue',x='reviewVisited')

pos = [ '2017-10-01', '2018-02-01', '2018-03-01', '2018-04-01', 
       '2018-05-01', '2018-06-01', '2018-07-01', '2018-08-01',
       '2018-09-01', '2018-10-01', '2018-11-01', '2018-12-01',
      '2019-01-01','2019-02-01','2019-03-01','2019-04-01','2019-05-01','2019-06-01','2019-07-01','2019-08-01','2019-09-01',
      '2019-10-01','2019-11-01','2019-12-01','2020-01-01','2020-02-01','2020-03-01','2020-04-01','2020-05-01','2020-06-01'
      ,'2020-07-01','2020-08-01','2020-09-01','2020-10-01','2020-11-01','2020-12-01','2021-01-01','2021-02-01','2021-03-01',
      '2021-04-01','2021-05-01','2021-06-01','2021-07-01','2021-08-01']
  
lab = [ 'Oct17', 'Feb18', 'Mar18', 'Apr18', 'May18', 'June18', 
       'July18', 'Aug18', 'Sept18', 'Oct18', 'Nov18', 'Dec18', 'Jan19',
      'Feb19', 'Mar19', 'Apr19', 'May19', 'June19', 
       'July19', 'Aug19', 'Sept19', 'Oct19', 'Nov19', 'Dec19','Jan20','Feb20', 'Mar20', 'Apr20', 'May20', 'June20', 
       'July20', 'Aug20', 'Sept20', 'Oct20', 'Nov20', 'Dec20','Jan21','Feb21', 'Mar21', 'Apr21', 'May21', 'June21', 
       'July21', 'Aug21']
  
plt.xticks( pos, lab)
plt.xticks(rotation=45)

# Easy to notice, both Portugal and rest of countries have had the same drop of ratings - meaning COVID has impacted probably all countries strongly


In [ ]:
# nr of unique users (user names) in the dataset
ds_one['userName'].nunique()

# from more than 90K records, it is clear that we have some users that have visited more than one of presented 100 attractions

These are the first observations from the exploration part. Yet, it is very important to say that these are very general and some of them confirmed our assumptions (like Covid has had a huge impact on all countries), but also some new like couples triptype has been by far the most popular type of vacation. 

After dataset's verification of quality and construction the data according our needs (for further analyses), we will present many more insights in regards to Portugal but also other, mostly neighbouring countries. Meaning, another "exploration" part (after data construction) will be unveiled. 

## Data verification of quality

In [ ]:
# Check of unique localIDs (resulting from describe report earlier)
# found 2 - genis and u which belong to MAG005 and MAG005 and so they will be transformed as such in Data preparation phase

ds_one.localID.unique()


In [ ]:
# checked missing values
ds_one.isnull().sum()


# Many missing values on triptypes and user location but we most likely keep all such observations (for some analyses they may be useful, and for those including such variables, they would be automatically excluded)
# At the moment, the ones from Name, country, ISO - dont really matter as another merge will need to be made

In [ ]:
# Finding duplicates now
ds_one.duplicated().sum()

# not duplicated rows, but lets check if same user name was not giving rating to same place twice

In [ ]:
ds_one[ds_one[['Name', 'userName', 'userLocation']].duplicated() == True]

# looks like as those, almost 7,5K rows are like that. In the data preparation phase, we only keep last records

In [ ]:
# check it via random username from the table
ds_one[ds_one["userName"] == 'michael.t@michaeltast']

# we can see that this user has the same user location twice. In fact, we would the only difference is in the extractionDate
# Anyways, we will remove these duplicates in the data preparation part

# DATA PREPARATION (Select, clean, construct, integrate, format)

## Data selection

In [ ]:
# displaying the dataset, that is going to be prepared for the modelling

ds_one.head(5)

# We go with the merged dataset - ds_one (attractions + reviews sheet merged together as up until now)
# We dont see any reasons (due to certain data limitations) to use the holidays dataset. It may be used once or twice ocassioanly, but not that the dataset would be 



## Data cleaning

In [ ]:
# The quality report unveiled 3 issues in the dataset: LocalID, Duplicated rows and Missing values
# First two we are going to clean/edit inmediately, starting with Local ID

ds_one['localID'] = ds_one['localID'].replace(['u'],'MAG006')
ds_one['localID'] = ds_one['localID'].replace(['genis'],'MAG005')

# However, we this change, we need to perform the merge operation again. TO be seen in the upcoming codes


In [ ]:
# Dropping dupplicated columns before another merge
ds_one = ds_one.drop(['Name', 'Country', 'ISO'], axis = 1)
ds_one.head(5)

# now the ds_one pretends to be like a "reviews dataset" from before which we need to merge it with the attractions dataset

In [ ]:
# FINAL Data collection followed by the data description 
ds = ds_one.merge(ds_attractions, on='localID', how='left')

In [ ]:
ds.describe(include='all').T

# description shows us improved values like having 100unique names that goes in hand with 100unique LocalIDs

In [ ]:
# erased all duplicates with same values for name, username and userlocation - only keeping the "last" record
ds = ds.drop_duplicates(['Name','userName','userLocation'],keep= 'last')
ds.info()

# now we can see that our dataset has lost more than 7K duplicated rows

In [ ]:
# check of no more duplicates acrss all potential variables but it looks all good now
ds[ds[['Name', 'userName', 'userLocation','tripType','reviewVisited','tripType','userContributions']].duplicated() == True]


In [ ]:
# crosscheck whether this is correct. We perform it on the same username via which we demonstrated the appearance of duplicates
ds[ds["userName"] == 'michael.t@michaeltast']

As previously stated, we are not going to do anything with the missing values yet, since they appear not to be harmful 

## Data Construct

In [ ]:
# Starting with the change the order --> of 3 merged columns to be placed at the begining after localID)
# + Getting rid of meaningless or duplicated columns (ISO, extractionDate, position on ranking, reviewlanguage, sitesonranking, TotalReviews)
ds = ds.loc[:, ["localID","Name","Country","globalRating","userName","userLocation","userContributions","tripType","reviewWritten","reviewVisited","reviewRating","reviewFullText"]]

# https://towardsdatascience.com/4-methods-for-changing-the-column-order-of-a-pandas-data-frame-a16cf0b58943



In [ ]:
ds.describe(include='all').T


General observations:

- 100 attractions visitied and kept in the data set from 25 countries
- Spain is te most visited and rated country, with Sagrada being the nr 1 attraction
- 3/4 of reviews are written are by people who have only rated once (1 location only)
- there are 12670 different user unique locations - Our team anticipates that it may be very hard to analyze this variable
- Out of the 2/3 of reviews written with certain trip type, couples are recorded in every second ocassion
- The month with the highest record of visitance and revie written in this date set is september 2019 (before covid)

Other:
- user contributions seem to have an outlier (looking at the Max value) - still will see if we will work with this variable

We are adding new columns, more specifically:
- Covid_time = after / before (based on the data already, but also overall start of the covid spread across Europe we set that everything before March2020 is the time before and everythig after --> after covid
- visits_together = each row with corresponding country will be assigned total number of visits per country (may be useful later in out analyses and comparison of Portugal to other countries
- Day of the week at which the review at certain attraction was given
- user visits for specific countries (explained in the report + to be understood from blocks of code later)

We also considered
- AVG time between attraction visit and review written but as one variable is with month and another with days, it would not be that accurate


In [ ]:
# Adding new column whether review visitied was before or after covid
ds.loc[ds['reviewVisited'] >= '2020-03-01', 'Covid_time'] = 'After Covid'
ds.loc[ds['reviewVisited'] < '2020-03-01', 'Covid_time'] = 'Before Covid'

In [ ]:
# Adding new column with total visits per each country 
ds['visits_together_per_country'] = ds['Country'].map(ds['Country'].value_counts())
ds.head(5)

# https://stackoverflow.com/questions/17709270/create-column-of-value-counts-in-pandas-dataframe

In [ ]:
# day of the week - review given
ds['day_of_week_review_given'] = ds['reviewWritten'].dt.day_name()
ds.head()

We also create separate dataset for:
- Portugal
- Spain
- Italy
- France
- Portugal + Italy + Spain + France as all these countries have high representation of records (more than 4000 each) and are most visited countries in Europe (source: https://www.schengenvisainfo.com/travel-guide/top-10-most-visited-european-countries/) - Portugal can be inspired and have the highest benchmark. Also culturally, these countries are the most similar ones. 

In [ ]:
# creation of mentioned dataset
Portugal = ds.apply(lambda row: row[ds['Country'].isin(['Portugal'])])
italy = ds.apply(lambda row: row[ds['Country'].isin(['Italy'])])
spain = ds.apply(lambda row: row[ds['Country'].isin(['Spain'])])
france = ds.apply(lambda row: row[ds['Country'].isin(['France'])])

pr_it_es_fr = ds.apply(lambda row: row[ds['Country'].isin(['Portugal','France','Italy','Spain'])])

In [ ]:
# dropping duplicates on username and user location (this one is not used in the analysis though)
Portugal_unique_visits = Portugal.drop_duplicates(['userName','userLocation'],keep= 'last')
italy_unique_visits = italy.drop_duplicates(['userName','userLocation'],keep= 'last')
spain_unique_visits = spain.drop_duplicates(['userName','userLocation'],keep= 'last')
france_unique_visits = france.drop_duplicates(['userName','userLocation'],keep= 'last')

In [ ]:
# adding user visits for all countries into separate columns
Portugal['user_visits_sum'] = Portugal['userName'].map(Portugal['userName'].value_counts())
italy['user_visits_sum'] = italy['userName'].map(italy['userName'].value_counts())
spain['user_visits_sum'] = spain['userName'].map(spain['userName'].value_counts())
france['user_visits_sum'] = france['userName'].map(france['userName'].value_counts())

In [ ]:
# merging dataset together
pr_it_es_fr_visits = [Portugal, italy, spain, france]
pr_it_es_fr_visits_final = pd.concat(pr_it_es_fr_visits)
pr_it_es_fr_visits_final

# https://towardsdatascience.com/joining-datasets-with-pythons-pandas-ed832f01450c

We believe that user location is a key variable to unveils some patters on portuguese visitors, yet as demonstrated earlier, there are so many unique values. Yet, for the portuguese dataset, we try to apply string contain method that could possibly group together majority of user location under common umbrella (like England, Scotland, UK --> all belong to UK). In many ocassions, there is a city and then the country stated (especially US), so we try to apply the method to have statisically significant data from updated user location that we can work with. 



In [ ]:
# For the mentioned experiment with the updated user location we will make a copy of a current Portuguese dataset
portugal_two = Portugal.copy()
portugal_two.isnull().sum()

# so we expect 641 records to have "missing location" string value under new column "updated_location"

In [ ]:
portugal_two['userLocation'] = portugal_two['userLocation'].fillna('Missing location - NaN')

# first we fill those missing values with "Missing location - NaN" string --> enabling us to these records write under updated location too

In [ ]:
# Creating new column on Portuguese dataset grouped updated location respectively. Those observations that do not match current value we expect to have "missing values"

portugal_two.loc[portugal_two['userLocation'].str.contains("UK|Ireland|United Kingdom|England"), 'Updated_location'] = 'United Kingdom'
portugal_two.loc[portugal_two['userLocation'].str.contains("US|United States|CA|ALA|AK|AS|AZ|AR|CA|CO|CT|DE|DC|FL|GA|HU|HI|ID|IL|IN|IA|KS|KY|LA|ME|MD|MA|MI|MN|MS|MO|MT|NE|NV|NH|NJ|NM|NY|NC|ND|MP|OH|OK|OR|PA|PR|RI|SC|SD|TN|TX|UM|UT|VT|VI|WA|VA|WV|WI|WY"), 'Updated_location'] = 'United States'
portugal_two.loc[portugal_two['userLocation'].str.contains("Portugal|portugal"), 'Updated_location'] = 'Portugal'
portugal_two.loc[portugal_two['userLocation'].str.contains("Canada|canada"), 'Updated_location'] = 'Canada'
portugal_two.loc[portugal_two['userLocation'].str.contains("Australia|New Zealand"), 'Updated_location'] = 'Australia/New Zealand'
portugal_two.loc[portugal_two['userLocation'].str.contains("Germany"), 'Updated_location'] = 'Germany'
portugal_two.loc[portugal_two['userLocation'].str.contains("Italy"), 'Updated_location'] = 'Italy'
portugal_two.loc[portugal_two['userLocation'].str.contains("Spain"), 'Updated_location'] = 'Spain'

portugal_two.loc[portugal_two['userLocation'].str.contains("Missing location - NaN"), 'Updated_location'] = 'Missing location - NaN'


# https://stackoverflow.com/questions/36653419/str-contains-to-create-new-column-in-pandas-dataframe
# https://stackoverflow.com/questions/26577516/how-to-test-if-a-string-contains-one-of-the-substrings-in-a-list-in-pandas


In [ ]:
portugal_two.isnull().sum()
# we have 1290 locations that have not been matched - so we will assign them a "different country" value

In [ ]:
portugal_two['Updated_location'] = portugal_two['Updated_location'].fillna('Different country')
portugal_two.isnull().sum()

# Again, triptyp missing values do not mind as 2/3 of the rows can be satisfactory when analysing triptypes separately

## Data Integration

No need to integrate any other data. We are good here.

## Data Format

Same as with Data integration, no formatting is needed for the analyses.

## DATA EXPLORATION PART 2 (to obtain insights from Portugal)

In [ ]:
# Histogram of Portugal 
# plt.figure(figsize=(15,7))
ax = sns.displot(data=portugal_two, x='reviewVisited', kind='hist', height=6, aspect=1.4, bins=35)


# setting customized ticklabels for x axis
pos = [ '2017-10-01', '2018-02-01', '2018-03-01', '2018-04-01', 
       '2018-05-01', '2018-06-01', '2018-07-01', '2018-08-01',
       '2018-09-01', '2018-10-01', '2018-11-01', '2018-12-01',
      '2019-01-01','2019-02-01','2019-03-01','2019-04-01','2019-05-01','2019-06-01','2019-07-01','2019-08-01','2019-09-01',
      '2019-10-01','2019-11-01','2019-12-01','2020-01-01','2020-02-01','2020-03-01','2020-04-01','2020-05-01','2020-06-01'
      ,'2020-07-01','2020-08-01','2020-09-01','2020-10-01','2020-11-01','2020-12-01','2021-01-01','2021-02-01','2021-03-01',
      '2021-04-01','2021-05-01','2021-06-01','2021-07-01','2021-08-01']
  
lab = [ 'Oct17', 'Feb18', 'Mar18', 'Apr18', 'May18', 'June18', 
       'July18', 'Aug18', 'Sept18', 'Oct18', 'Nov18', 'Dec18', 'Jan19',
      'Feb19', 'Mar19', 'Apr19', 'May19', 'June19', 
       'July19', 'Aug19', 'Sept19', 'Oct19', 'Nov19', 'Dec19','Jan20','Feb20', 'Mar20', 'Apr20', 'May20', 'June20', 
       'July20', 'Aug20', 'Sept20', 'Oct20', 'Nov20', 'Dec20','Jan21','Feb21', 'Mar21', 'Apr21', 'May21', 'June21', 
       'July21', 'Aug21']
  
plt.xticks( pos, lab)
plt.xticks(rotation=90)
plt.show()


# Clearly seen a huuge drop after March 2020 to almost zero.
# Drop after july can explain long term vacations (more than one months form which many may be assigned to July - same happened in Spain and can be observed via density plots below)
# lineplot - to explore what has happened between 
# https://seaborn.pydata.org/generated/seaborn.lineplot.html perhaps based on pivot table

In [ ]:
fig, ax = plt.subplots(figsize=(15,7))

sns.kdeplot(portugal_two[portugal_two['Name']=='Torre de Belém']['reviewVisited'], shade=False, color='green', x='reviewVisited')
sns.kdeplot(portugal_two[portugal_two['Name']=='Quinta da Regaleira']['reviewVisited'], shade=False, color='blue',x='reviewVisited')
sns.kdeplot(portugal_two[portugal_two['Name']=='Ponte de Dom Luís I']['reviewVisited'], shade=False, color='pink',x='reviewVisited')
sns.kdeplot(portugal_two[portugal_two['Name']=='Park and National Palace of Pena']['reviewVisited'], shade=False, color='black',x='reviewVisited')
sns.kdeplot(portugal_two[portugal_two['Name']=='Mosteiro dos Jeronimos']['reviewVisited'], shade=False, color='red',x='reviewVisited')
sns.kdeplot(portugal_two[portugal_two['Name']=='Cais da Ribeira']['reviewVisited'], shade=False, color='yellow',x='reviewVisited')
sns.kdeplot(portugal_two[portugal_two['Name']=='Bom Jesus do Monte']['reviewVisited'], shade=False, color='purple',x='reviewVisited')


# Allmost every attracrtion had same distribution relative to their own count of ratings across the time (outstanding is just cais de ribeira - timeout market)



In [ ]:
# DENSITY PLOT (Kernel Density Estimate) Portugal vs its main competitors (spain, italy, france)

fig, ax = plt.subplots(figsize=(15,7))

# fig, ax = plt.subplots(figsize=(15,7))

sns.kdeplot(pr_it_es_fr[pr_it_es_fr['Country']=='Portugal']['reviewVisited'], shade=False, color='green', x='reviewVisited')
sns.kdeplot(pr_it_es_fr[pr_it_es_fr['Country']=='Spain']['reviewVisited'], shade=False, color='blue',x='reviewVisited')
sns.kdeplot(pr_it_es_fr[pr_it_es_fr['Country']=='Italy']['reviewVisited'], shade=False, color='pink',x='reviewVisited')
sns.kdeplot(pr_it_es_fr[pr_it_es_fr['Country']=='France']['reviewVisited'], shade=False, color='black',x='reviewVisited')

# even though there may be a small drops in case of Portugal and Spain, the trendline is almost the same for each country, meaning each of the countries have been hit almost the same by Covid19 and it is brutally seen through histograms and density plots presented

### DATA EXPLORATION - INSIGHT 1 (PRT vs other countries - covid impact on increase of ratings (visitance))

In [ ]:
# AVG rating before and after covid in Portugal
pt_crosstab_ratings = pd.crosstab(index=portugal_two['Name'],
            columns=portugal_two['Covid_time'],
            values=portugal_two['reviewRating'],
            aggfunc=np.mean)
pt_crosstab_ratings.style.background_gradient(axis=None, low=0.75, high=1.0)


# even some nice differences may be found, bear in mind number of observations after covid and whether they can be statiscally significant

# https://pandas.pydata.org/docs/reference/api/pandas.io.formats.style.Styler.background_gradient.html
# comparing mean of the ratings for each of the attractions before and after covid 

In [ ]:
# numeric impact on each attraction before and after covid 

pt_crosstab = pd.crosstab(index=portugal_two['Name'],
            columns=portugal_two['Covid_time'],
            values=portugal_two['reviewRating'],
            aggfunc="count")
pt_crosstab.style.background_gradient(axis=None, low=0.75, high=1.0)

In [ ]:
# numeric impact on each attraction before and after - but now os % increase, to see which attractions seem to "suffer" less

pt_crosstab["AC growth after before covid"] = (pt_crosstab["After Covid"] / pt_crosstab["Before Covid"]) * 100
pt_crosstab

In [ ]:
# seeing absolute number of visit before and after covid
pr_es_it_fr_crosstab = pd.crosstab(index=pr_it_es_fr['Country'],
            columns=pr_it_es_fr['Covid_time'],
            values=pr_it_es_fr['reviewRating'],
            aggfunc="count")

pr_es_it_fr_crosstab["AC growth after before covid"] = (pr_es_it_fr_crosstab["After Covid"] / pr_es_it_fr_crosstab["Before Covid"]) * 100
pr_es_it_fr_crosstab

In [ ]:
# building DF to display AC growth after before covid

France = (223/3403) * 100
print(France)

Italy = (427/5859) * 100
print(Italy)

portugal = (309/4948) * 100
print(portugal)

Spain =  698/19752 * 100
print(Spain)

together_increase = [['Italy', 7.287933094384708], ['France', 6.553041434028799], ['portugal', 6.2449474535165725], ['Spain', 3.5338193600648036]]
 
# Create the pandas DataFrame
together_increase_ac = pd.DataFrame(together_increase, columns = ['Country', 'increase after covid (%)'])
 
# print dataframe.

In [ ]:
# Bar chart representation of ratings growth after covid

together_increase_ac.plot(kind="bar", x='Country', y='increase after covid (%)');
plt.title('% of visitance after covid (compared to before covid)')
ax.set_xticklabels(ax.get_xticklabels(), rotation=90, ha="right")
plt.tight_layout()
plt.show()

# clearly, Spain has decreased a lot!

In [ ]:
# fig_dims = (6, 4)
# fig, ax = plt.subplots(figsize=fig_dims)

order_country = ["Italy", "France", "portugal","Spain"]
x = sns.catplot(x="Country", y="increase after covid (%)", kind="bar", data=together_increase_ac, order=order_country, ax=ax)
g.fig.suptitle("% of visitance after covid (compared to before covid)", y=1.05)
g.set(xlabel="Country", ylabel="Increase after covid")
sns.set_context("notebook")






plt.show()

### DATA EXPLORATION - INISGHT 2 (proportion of visits in Portugal and Spain is far better than in France, Italy)

In [ ]:
# proportion of visitors based on how many visits hey have made in the given country 

nr_of_visits_crosstab_country = pd.crosstab(pr_it_es_fr_visits_final['Country'],pr_it_es_fr_visits_final['user_visits_sum'],normalize='index')

nr_of_visits_crosstab_country = nr_of_visits_crosstab_country.div(nr_of_visits_crosstab_country.sum(1), axis=0)

# Draw
fig, ax = plt.subplots(figsize=(10,5))
g = nr_of_visits_crosstab_country.plot(kind='barh', stacked=True, ax=ax)





In [ ]:
# same just in portugal across attractions 

nr_of_visits_crosstab_pt_attractions = pd.crosstab(Portugal['Name'],Portugal['user_visits_sum'],normalize='index')

nr_of_visits_crosstab_pt_attractions = nr_of_visits_crosstab_pt_attractions.div(nr_of_visits_crosstab_pt_attractions.sum(1), axis=0)

# Draw
fig, ax = plt.subplots(figsize=(14,5))
g = nr_of_visits_crosstab_pt_attractions.plot(kind='barh', stacked=True, ax=ax)



### DATA EXPLORATION - INISGHT 3 (Couples being by far the most popular triptypes)

In [ ]:
# Draw triptypes in Portugal in ascending value
plt.figure(figsize=(14,7))
ax = sns.countplot(x='tripType', data=portugal_two, order=pd.value_counts(portugal_two['tripType']).iloc[:20].index);
plt.title('Most tripType in Portugal by type')
ax.set_xticklabels(ax.get_xticklabels(), rotation=90, ha="right")
plt.tight_layout()
plt.show()

In [ ]:
triptype_pt = pd.crosstab(portugal_two['Name'],portugal_two['tripType'],normalize='index')

triptype_pt = triptype_pt.div(triptype_pt.sum(1), axis=0)

# Draw
fig, ax = plt.subplots(figsize=(12,8))
g = triptype_pt.plot(kind='barh', stacked=True, ax=ax)

# clearly couples is the major representation of triptype across all attractions in Poprtugal

In [ ]:
# triptypes proportion across main countries 

triptype_all = pd.crosstab(pr_it_es_fr['Country'],pr_it_es_fr['tripType'],normalize='index')

triptype_all = triptype_all.div(triptype_all.sum(1), axis=0)

# Draw
fig, ax = plt.subplots(figsize=(12,8))
g = triptype_all.plot(kind='barh', stacked=True, ax=ax)

# triptypes across main countries dont differ too much, maybe families are less representated in Portugal than in other countries

In [ ]:
# density plot in Portugal based on different triptypes
fig, ax = plt.subplots(figsize=(15,7))


sns.kdeplot(portugal_two[portugal_two['tripType']=='Couples']['reviewVisited'], shade=False, color='blue')
sns.kdeplot(portugal_two[portugal_two['tripType']=='Friends']['reviewVisited'], shade=False, color='red')
sns.kdeplot(portugal_two[portugal_two['tripType']=='Family']['reviewVisited'], shade=False, color='green')
sns.kdeplot(portugal_two[portugal_two['tripType']=='Solo']['reviewVisited'], shade=True, color='pink')

# Shaded colour is trip type - solo --> proving, this has been the least hit triptype by the covid, yet also very much. However, it confirms assumption of which trip type has beeb affected the least.
# Covid clearly hasn't much impacted different triptypes, only solo traveling which makess sense

### DATA EXPLORATION - INSIGHT 3 - Portuguese as only tourisits (or mayority) of tourists in Portugal after Covid 

In [ ]:
# density plot based on different updated_location (user location) in Portugal
fig, ax = plt.subplots(figsize=(15,7))

# sns.kdeplot(portugal_two[portugal_two['Updated_location']=='United States']['reviewVisited'], shade=False, color='yellow', x='reviewVisited')
# sns.kdeplot(portugal_two[portugal_two['Updated_location']=='United Kingdom']['reviewVisited'], shade=False, color='red', x='reviewVisited')
sns.kdeplot(portugal_two[portugal_two['Updated_location']=='Canada']['reviewVisited'], shade=False, color='green', x='reviewVisited')
sns.kdeplot(portugal_two[portugal_two['Updated_location']=='Portugal']['reviewVisited'], shade=True, color='blue',x='reviewVisited')
sns.kdeplot(portugal_two[portugal_two['Updated_location']=='Spain']['reviewVisited'], shade=False, color='pink',x='reviewVisited')
sns.kdeplot(portugal_two[portugal_two['Updated_location']=='Australia/New Zealand']['reviewVisited'], shade=False, color='black',x='reviewVisited')
sns.kdeplot(portugal_two[portugal_two['Updated_location']=='Different country']['reviewVisited'], shade=False, color='yellow',x='reviewVisited')
sns.kdeplot(portugal_two[portugal_two['Updated_location']=='Missing location - NaN']['reviewVisited'], shade=False, color='purple',x='reviewVisited')

pos = [ '2017-10-01', '2018-02-01', '2018-03-01', '2018-04-01', 
       '2018-05-01', '2018-06-01', '2018-07-01', '2018-08-01',
       '2018-09-01', '2018-10-01', '2018-11-01', '2018-12-01',
      '2019-01-01','2019-02-01','2019-03-01','2019-04-01','2019-05-01','2019-06-01','2019-07-01','2019-08-01','2019-09-01',
      '2019-10-01','2019-11-01','2019-12-01','2020-01-01','2020-02-01','2020-03-01','2020-04-01','2020-05-01','2020-06-01'
      ,'2020-07-01','2020-08-01','2020-09-01','2020-10-01','2020-11-01','2020-12-01','2021-01-01','2021-02-01','2021-03-01',
      '2021-04-01','2021-05-01','2021-06-01','2021-07-01','2021-08-01']
  
lab = [ 'Oct17', 'Feb18', 'Mar18', 'Apr18', 'May18', 'June18', 
       'July18', 'Aug18', 'Sept18', 'Oct18', 'Nov18', 'Dec18', 'Jan19',
      'Feb19', 'Mar19', 'Apr19', 'May19', 'June19', 
       'July19', 'Aug19', 'Sept19', 'Oct19', 'Nov19', 'Dec19','Jan20','Feb20', 'Mar20', 'Apr20', 'May20', 'June20', 
       'July20', 'Aug20', 'Sept20', 'Oct20', 'Nov20', 'Dec20','Jan21','Feb21', 'Mar21', 'Apr21', 'May21', 'June21', 
       'July21', 'Aug21']
  
plt.xticks( pos, lab)
plt.xticks(rotation=45)

### DATA EXPLORATION - other observations

In [ ]:
plt.figure(figsize=(15,7))
ax = sns.countplot(y='day_of_week_review_given', data=portugal_two, order=pd.value_counts(portugal_two['day_of_week_review_given']).iloc[:7].index);

# On sundays, people give most ratings but the difference between days is not so much different one from another. Lets have a look into different "ratings" habit - to see if we can find some patterns (like on friday, people tend to give better ratings because they feel better ahead of weekend)


In [ ]:
# AVG rating in Portugal across days when rating is given

pt_crosstab_two = pd.crosstab(index=portugal_two['Country'],
            columns=portugal_two['day_of_week_review_given'],
            values=portugal_two['reviewRating'],
            aggfunc=np.mean)
pt_crosstab_two.style.background_gradient(axis=None, low=0.75, high=1.0)

In [ ]:

# AVG rating across days when rating is given, segmented by countries

countries_crosstab_two = pd.crosstab(index=pr_it_es_fr['Country'],
            columns=pr_it_es_fr['day_of_week_review_given'],
            values=pr_it_es_fr['reviewRating'],
            aggfunc=np.mean)
countries_crosstab_two.style.background_gradient(axis=None, low=0.75, high=1.0)

# Modeling

## Modeling - Selecting modeling technique

In [ ]:
# finding out how many users have visitied at least 2 attractions in Portugal - in case there is a high number, we can run association rules
Portugal_two_visits = portugal_two[portugal_two['userName'].map(portugal_two['userName'].value_counts()) > 1]
Portugal_two_visits['userName'].nunique()

# 959 is great number, telling is there is at least 959 combinations of 2+ attractions in the dataset - this is statiscally significant for a try of a association rules!

we only will go with association rules, also RFM and similarity matrixes are done, however not used in the report and neither would be used for the final presentation to the stakeholders

## Association rules

In [ ]:
portugal_associations = portugal_two[['Name', 'userName']]
portugal_associations

In [ ]:
# pivot table with only portuguese attractions! 
portugal_associations_pivot = pd.pivot_table(portugal_associations[['userName', 'Name']], index='userName', columns='Name', aggfunc=lambda x: 1 if len(x)>0 else 0).fillna(0)
portugal_associations_pivot

# pivot table with all user names --> 1 means the given user visited/gave rating to such place, if 0, it means that no

In [ ]:
portugal_frequent_namesets = apriori(portugal_associations_pivot, min_support=0.01, use_colnames=True)

In [ ]:
# Generate the association rules - by support
rulesSupport = association_rules(portugal_frequent_namesets, metric="support", min_threshold=0.001)
rulesSupport.sort_values(by='support', ascending=False, inplace=True)
rulesSupport.head(10)

# only torre de belem together with mosteiro de jeronimos are present in 10% of occasions. Other combinations very low


In [ ]:
# Generate the association rules - by confidence
rules_confidence = association_rules(portugal_frequent_namesets, metric="confidence", min_threshold=0.001)
rules_confidence.sort_values(by='confidence', ascending=False, inplace=True)
rules_confidence.head(10)

# now we get to see which once are those ones with higher confidence
# Interesting to see, antecedent support is really low (between 2-4%) and those match with either Torre de Belem or Mosteiro whose appeearance in dataset is much more frequent

# SO FAR no big insights unveiled, we have suspicion of having dataset that is not representative enoguh

In [ ]:
# Generate the association rules - by lift
rules_lift = association_rules(portugal_frequent_namesets, metric="lift", min_threshold=0.001)
rules_lift.sort_values(by='lift', ascending=False, inplace=True)
rules_lift.head(10)

# not statiscally god enough either :(

In [ ]:
# Add a column with the length and then displaying combinations that have appeared together in more than 5% of ocassions
portugal_frequent_namesets['length'] = portugal_frequent_namesets['itemsets'].apply(lambda x: len(x))
# Length=2 and Support>=0.05
portugal_frequent_namesets[(portugal_frequent_namesets['length'] == 2) & (portugal_frequent_namesets['support'] >= 0.05)]

# nothing breaking is unveiled, these combinations make sense as attractions are close one to another

In [ ]:
# Scatter plot of these rules

# Draw
fig, ax = plt.subplots(figsize=(8,6))
g = sns.scatterplot(data=rulesSupport, x="support", y="confidence", size="lift", sizes=(20, 200), alpha=0.8)

# Decoration
sns.despine()
plt.title("Rules with support above 10% (Lift as size)", fontsize=plots_Title_fontSize)
plt.xlabel("Support")
plt.ylabel("Confidence")
plt.rc('axes', labelsize=subPlots_label_fontSize)
handles, labels = ax.get_legend_handles_labels()
ax.legend(handles=handles, labels=labels, loc='upper center', 
          ncol=6, bbox_to_anchor=(0.45, 1.03), frameon=False)

# clearly seen 2 patterns
# First is as already illustrated that only 2 combinations (of 2 attractions) have support (presence in the dataset) more than 5% - but they also have lower confidence
# there are some combinations with low support but high degree of confidence
# on the left bottom corner, there are many unrelated attractions combinations

In [ ]:
# Bubble plot of the rules based on confidence

# Replace frozen sets with strings
rules_confidence['antecedents_'] = rules_confidence['antecedents'].apply(lambda a: ','.join(list(a)))
rules_confidence['consequents_'] = rules_confidence['consequents'].apply(lambda a: ','.join(list(a)))

# Draw
fig, ax = plt.subplots(figsize=(13,7))
ax= plt.scatter(data=rules_confidence, x='consequents_', y='antecedents_', s = rules_confidence['lift']*20, edgecolors = "red", c = "white", zorder = 2)

# Decoration
nRules=rules_confidence.shape[0]
plt.title(f"Grouped matrix of the {nRules} rules", fontsize=plots_Title_fontSize)
plt.xlabel("Consequents (RHS)")
plt.ylabel("Antecedents (LHS)")
plt.grid(ls = "--", zorder = 1)
fig.autofmt_xdate()
plt.tight_layout()
plt.xticks(rotation=45)

IMPORTANT! FOLLOWING TWO MODELINGs DONT BELONG TO THE REPORT BUT ARE PRESENTED!

## RFM modeling

In [ ]:
dateMax = portugal_two.reviewVisited.max()


In [ ]:
# need to add column = totals of visits per each username 
portugal_rfm = portugal_two.copy()
portugal_rfm['user_visits_sum'] = portugal_rfm['userName'].map(portugal_rfm['userName'].value_counts())
portugal_rfm.head(5)

In [ ]:
X = portugal_rfm.groupby(['userName']).agg(Recency=('reviewVisited', lambda date: (dateMax - date.max()).days),
                                   Frequency=('user_visits_sum', 'max'),
                                   Monetary=('userContributions', 'max')).fillna(0)

In [ ]:
X.head(5)

In [ ]:
# Determinining RFM cutting points based on the quartiles (default for pandas "describe")
cols = ['Recency','Frequency','Monetary']
table = X[cols].describe()
table

# clear outliers on monetary max
# need to investigate recency

In [ ]:
# Calculate RMF scores

# Function
def RFMScore(x, col):
    if x <= X.quantile(0.25)[col]:
        return '1'
    elif x <= X.quantile(0.5)[col]:
        return '2'
    elif x <= X.quantile(0.75)[col]:
        return '3'
    else:
        return '4'

# Process
X['RScore'] = X['Recency'].apply(RFMScore, col='Recency')
X['FScore'] = X['Frequency'].apply(RFMScore, col='Frequency')
X['MScore'] = X['Monetary'].apply(RFMScore, col='Monetary')

In [ ]:
X.head(60)

In [ ]:
# Create a column with full RMF score and sort the results

# Transform to string
cols = ['RScore','FScore','MScore']
X[cols] = X[cols].astype(str)

# Concatenate
X['RFMScore'] = X['RScore'] + X['FScore'] + X['MScore']

# Sort
X = X.sort_values(by=['RFMScore'])
X.head(5)

In [ ]:
# Calculate statistics per RFM segment
RFMStats = X.reset_index().groupby(['RFMScore']).agg(NrUsers=('userName', lambda i: len(i.unique())),
                                                     avgRecency=('Recency', 'mean'),
                                                     avgFrequency=('Frequency', 'mean'),
                                                     avgMonetary=('Monetary', 'mean')).fillna(0)
RFMStats.head(30)

In [ ]:
X[X['RFMScore']=='144']

In [ ]:
# Histogram of RFM
%matplotlib inline
cols = ['Recency','Frequency','Monetary']

# Draw
fig, ax = plt.subplots(1, 3, figsize=(10,4))
for var, subplot in zip(X[cols], ax.flatten()):
    g = sns.histplot(data=X,
                bins=10,
                 x=var,
                 ax=subplot,
                 kde=False)

# Decoration
sns.despine()
plt.rc('axes', labelsize=subPlots_label_fontSize)
fig.suptitle("RFM histograms", fontsize=plots_Title_fontSize);

# clear to see that that modeling dataset should have been cleaned by the outliers (monetary for ex. which is user contributions)
# also distribution shows that majority of values are close to each other and so, the mean representation of each segment might not differ much, making the analysis 

In [ ]:
# Treemap with number of customers by segment

# Define colors for levels
def assignColor(rfm):
    if (rfm=='144'):
        hex='#D7D7D7'   # Platinium
    elif (rfm in ['142','143','133','134','124']):
        hex='#C9B037'   # Gold
    elif (rfm in ['141','131','132','122','123','113','114']):
        hex='#B4B4B4'   # Silver
    else:
        hex='#6A3805'   # Bronze
    return hex

color = [assignColor(x) for x in RFMStats.index]

# Draw
fig, ax = plt.subplots(figsize=(10,7))

# Plot
squarify.plot(sizes=RFMStats['NrUsers'], 
              label=RFMStats.index,
              color = color,
              alpha=.9,
              pad=True)                    

# Decoration
plt.title("Number of customers by RFM segment",fontsize=plots_Title_fontSize)
plt.axis('off')
plt.show()

In [ ]:
# RFM Heatmap

# Prepare data
tempDF = RFMStats
tempDF['Frequency'] = tempDF.index.str[1]
tempDF['Monetary'] = tempDF.index.str[2]
pt = pd.pivot_table(tempDF, values='avgRecency', 
                     index=['Frequency'], 
                     columns='Monetary')

# Draw
fig , ax = plt.subplots(figsize=(6, 8))
heatmap = sns.heatmap(pt,
                      square = True,
                      linewidths = .5,
                      cmap = 'Blues',
                      cbar=False,
                      fmt='.0f',
                      annot = True,
                      annot_kws = {'size': heatmaps_text_fontSize+2})

# Decoration
plt.title("Average Recency (days) by Monetary and Frequency levels", fontsize=plots_Title_fontSize)
sns.set_style({'xtick.bottom': True}, {'ytick.left': True})

## SIMILARITIES / DISIMILARITIES

In [ ]:
portugal_two_pivot_table = pd.pivot_table(portugal_two[['userName', 'Name']], index='userName', columns='Name', aggfunc=lambda x: 1 if len(x)>0 else 0).fillna(0)
portugal_two_pivot_table.shape

In [ ]:
# customer similarity matrix
customer_customer_sim_matrix = pd.DataFrame(
    pairwise_distances(portugal_two_pivot_table,metric='cosine'),
    columns = portugal_two_pivot_table.index,
    index = portugal_two_pivot_table.index
)
customer_customer_sim_matrix = customer_customer_sim_matrix.apply(lambda x: 1-x, axis=1) # Transform dissimilarity to similarity
customer_customer_sim_matrix

In [ ]:
# similarity matrix across attractions
product_product_sim_matrix = pd.DataFrame(
    pairwise_distances(portugal_two_pivot_table.T,metric='cosine'),
    columns = portugal_two_pivot_table.columns,
    index = portugal_two_pivot_table.columns
    )
product_product_sim_matrix = product_product_sim_matrix.apply(lambda x: 1-x, axis=1) # Transform dissimilarity to similarity
product_product_sim_matrix

# again, the biggest similarity is between Torre de Belem and Mosteiro
# other cna be Ponde de Dom Luis and Cais de Riberia
# Are here any marketing recommendations based on such analysis???

In [ ]:
top_10_similar_items = list(
    product_product_sim_matrix
        .loc['Park and National Palace of Pena']
        .sort_values(ascending=False)
        .iloc[1:4]         # 1 to 11 instead of 0 to 10 because the first is the product itself
    .index
)
top_10_similar_items
# Based on similarity - once we pick one attraction, the top 3 most similar attractiosn can be spitted out - again, useful for some marketing activities?
# this method can be used for collective tickets to the attractions (BCN example with museums) maybe?